# Generate MGRS Grid Geoparquet

Official Grid (used for Sentinel-2 tiling) from 100MB KML:
https://sentinel.esa.int/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml 

Conver to [geoparquet 1.0](https://github.com/opengeospatial/geoparquet) for easier visualization and batch processing of other datasets (e.g. Sentinel-1)

In [1]:
import geopandas as gpd
import dask_geopandas
import shapely
import re

In [2]:
%load_ext watermark
%watermark --iversions

shapely       : 1.8.4
dask_geopandas: 0.2.0
re            : 2.2.1
geopandas     : 0.11.1



In [3]:
!wget -nc https://sentinel.esa.int/documents/247904/1955685/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml 

File ‘S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml’ already there; not retrieving.



In [4]:
!ogrinfo -so S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml Features

INFO: Open of `S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'
      using driver `LIBKML' successful.

Layer name: Features
Geometry: Unknown (any)
Feature Count: 56686
Extent: (-180.000000, -83.835951) - (180.000000, 84.644279)
Layer SRS WKT:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Name: String (0.0)
description: String (0.0)
timestamp: DateTime (0.0)
begin: DateTime (0.0)
end: DateTime (0.0)
altitudeMode: String (0.0)
tessellate: Integer (0.0)
extrude: Inte

In [5]:
# DIM=2 drops Z dimension
!ogr2ogr -overwrite -dim 2 -skipfailures /tmp/tmp.geojson S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml Features

In [6]:
gf = gpd.read_file('/tmp/tmp.geojson') #GEOMETRYCOLLECTION Z  
gf

,Name,description,tessellate,extrude,visibility,snippet,geometry
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 -73.05...
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 -83.80...
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 -82.91...
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 -82.01...
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 -81.12...
...,...,...,...,...,...,...,...
56681,60XWM,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 77.364...
56682,60XWN,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 78.260...
56683,60XWP,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 79.156...
56684,60XWQ,TILE PROPERTIES<br><table border=0 cellpadding...,-1,0,-1,,GEOMETRYCOLLECTION (POLYGON ((180.00000 80.051...


In [7]:
# GEOMETRYCOLLECTION contains polygon and centroid. If polygons cross antimeridian split into 2
# Assume first geometry is main footprint polygon:
gf['geometry'] = gf.geometry.apply(lambda x: x.geoms[0])

In [8]:
# NOTE very robust functions to extract from HTML
def get_utm_wkt(row):
    text = row.description.split('<b>')[-2] #UTM
    m = re.findall('MULTIPOLYGON\(\(\((.+?)\)\)\)', text)
    UTM_WKT = f'POLYGON (({m[0]}))'
    return UTM_WKT


def get_epsg(row):
    text = row.description.split('<b>')[2] #EPSG
    m = re.findall('<font COLOR="#008000">(.+?)</font>', text)
    EPSG = int(m[0])
    return EPSG

In [9]:
# Add UTM_WKT and EPSG from "Description" HTML
gf['epsg'] = gf.apply(get_epsg, axis=1)
gf['utm_wkt'] = gf.apply(get_utm_wkt, axis=1)

In [10]:
gf = gf.drop(columns=['description','tessellate','extrude','visibility','snippet'])
gf.head()

,Name,geometry,epsg,utm_wkt
0,01CCV,"POLYGON ((180.00000 -73.05974, 176.86462 -72.9...",32701,"POLYGON ((300000 2000020,300000 1890220,409800..."
1,01CDH,"POLYGON ((180.00000 -83.80855, 174.71288 -83.7...",32701,"POLYGON ((399960 800020,399960 690220,509760 6..."
2,01CDJ,"POLYGON ((180.00000 -82.91344, 175.74819 -82.8...",32701,"POLYGON ((399960 900040,399960 790240,509760 7..."
3,01CDK,"POLYGON ((180.00000 -82.01866, 176.55270 -81.9...",32701,"POLYGON ((399960 1000000,399960 890200,509760 ..."
4,01CDL,"POLYGON ((180.00000 -81.12317, 177.19616 -81.1...",32701,"POLYGON ((399960 1100020,399960 990220,509760 ..."


In [11]:
land = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [12]:
%%time 

# Keep only tiles containing land
dask_df = dask_geopandas.from_geopandas(gf, npartitions=4)
ind_land = dask_df.intersects(land.unary_union).compute()

CPU times: user 3min 58s, sys: 175 ms, total: 3min 59s
Wall time: 1min


In [13]:
gfL = gf[ind_land].reset_index(drop=True)
gfL

,Name,geometry,epsg,utm_wkt
0,01KAB,"POLYGON ((180.00000 -17.25048, 179.23921 -17.2...",32701,"POLYGON ((99960 8200000,99960 8090200,209760 8..."
1,01LAC,"POLYGON ((180.00000 -16.34738, 179.25695 -16.3...",32701,"POLYGON ((99960 8300020,99960 8190220,209760 8..."
2,01LBC,"POLYGON ((-179.79445 -15.35907, -178.77220 -15...",32701,"POLYGON ((199980 8300020,199980 8190220,309780..."
3,01VCK,"POLYGON ((180.00000 62.10763, 179.16853 62.090...",32601,"POLYGON ((300000 7000020,300000 6890220,409800..."
4,01VCL,"POLYGON ((180.00000 63.00584, 179.05150 62.986...",32601,"POLYGON ((300000 7100040,300000 6990240,409800..."
...,...,...,...,...
19038,60WWE,"POLYGON ((180.00000 71.09077, 176.99945 71.115...",32660,"POLYGON ((499980 8000040,499980 7890240,609780..."
19039,60WWS,"POLYGON ((176.99958 64.92414, 179.32025 64.906...",32660,"POLYGON ((499980 7200000,499980 7090200,609780..."
19040,60WWT,"POLYGON ((176.99956 65.82155, 179.40072 65.802...",32660,"POLYGON ((499980 7300020,499980 7190220,609780..."
19041,60WWU,"POLYGON ((176.99955 66.71886, 179.48759 66.699...",32660,"POLYGON ((499980 7400040,499980 7290240,609780..."


In [14]:
# Rename
gfL.rename(columns=dict(Name='tile'), inplace=True)

In [15]:
# Add simple UTM bounds (left, down, right, up)
gfL['utm_bounds'] = gfL.utm_wkt.apply(lambda x: shapely.wkt.loads(x).bounds).astype(str)

In [16]:
gfL.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19043 entries, 0 to 19042
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   tile        19043 non-null  object  
 1   geometry    19043 non-null  geometry
 2   epsg        19043 non-null  int64   
 3   utm_wkt     19043 non-null  object  
 4   utm_bounds  19043 non-null  object  
dtypes: geometry(1), int64(1), object(3)
memory usage: 744.0+ KB


In [17]:
gfL.head()

,tile,geometry,epsg,utm_wkt,utm_bounds
0,01KAB,"POLYGON ((180.00000 -17.25048, 179.23921 -17.2...",32701,"POLYGON ((99960 8200000,99960 8090200,209760 8...","(99960.0, 8090200.0, 209760.0, 8200000.0)"
1,01LAC,"POLYGON ((180.00000 -16.34738, 179.25695 -16.3...",32701,"POLYGON ((99960 8300020,99960 8190220,209760 8...","(99960.0, 8190220.0, 209760.0, 8300020.0)"
2,01LBC,"POLYGON ((-179.79445 -15.35907, -178.77220 -15...",32701,"POLYGON ((199980 8300020,199980 8190220,309780...","(199980.0, 8190220.0, 309780.0, 8300020.0)"
3,01VCK,"POLYGON ((180.00000 62.10763, 179.16853 62.090...",32601,"POLYGON ((300000 7000020,300000 6890220,409800...","(300000.0, 6890220.0, 409800.0, 7000020.0)"
4,01VCL,"POLYGON ((180.00000 63.00584, 179.05150 62.986...",32601,"POLYGON ((300000 7100040,300000 6990240,409800...","(300000.0, 6990240.0, 409800.0, 7100040.0)"


In [18]:
# Save as geoparquet instead
gfL.to_parquet('MGRS_LAND.parquet')